In [15]:
import math 
import numpy as np
import plotly.graph_objects as go
from utils import basic_dict, read_db

volume = 80 # Abschaetzung eines normal grossen Raumes


surface = {'wall1': 20, 'wall2': 20, 'wall3': 10, 'wall4': 10, 'wall5': 8, 'wall6': 8} # Abschaetzung eines normal grossen Raumes

alpha = {'125 Hz': [0.2, 0.1 ,0.2, 0.1, 0.1, 0.1, 0.1], 
        '250 Hz': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 
        '500 Hz': [0.02, 0.1, 0.2, 0.07, 0.1, 0.2, 0.1],
        '1 kHz': [0.1, 0.1, 0.08, 0.1, 0.1, 0.1, 0.1], 
        '2 kHz': [0.1, 0.04, 0.1, 0.2, 0.1, 0.1, 0.1], 
        '4 kHz': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
        } 

sub_surface = {'wall1':[5, 4], 'wall2':[6, 2], 'wall3': [5], 'wall4': [], 'wall5': [], 'wall6': [1, 2, 3]}

sub_alpha = {'125 Hz': {'wall1':[0.15, 0.36], 'wall2':[0.16, 0.12], 'wall3': [0.13], 'wall4': [], 'wall5': [], 'wall6': [0.1, 0.2, 0.3]},
            '250 Hz': {'wall1':[0.2, 0.1], 'wall2':[0.3, 0.1], 'wall3': [0.13], 'wall4': [], 'wall5': [], 'wall6': [0.1, 0.2, 0.3]}, 
            '500 Hz': {'wall1':[0.2, 0.1], 'wall2':[0.2, 0.1], 'wall3': [0.13], 'wall4': [], 'wall5': [], 'wall6': [0.1, 0.2, 0.3]}, 
            '1 kHz' : {'wall1':[0.2, 0.1], 'wall2':[0.2, 0.1], 'wall3': [0.13], 'wall4': [], 'wall5': [], 'wall6': [0.1, 0.2, 0.3]},
            '2 kHz':  {'wall1':[0.2, 0.1], 'wall2':[0.2, 0.1], 'wall3': [0.13], 'wall4': [], 'wall5': [], 'wall6': [0.1, 0.2, 0.3]},
            '4 kHz':  {'wall1':[0.2, 0.1], 'wall2':[0.2, 0.1], 'wall3': [0.13], 'wall4': [], 'wall5': [], 'wall6': [0.1, 0.2, 0.3]}  
            }       

use = 'Musik'

class room: 
    '''Class inheriting all functions for calculations and making plots.'''

    def __init__(self, volume, surface, sub_surface, alpha, sub_alpha, use):
        '''Function to initialize the class "room"'''
        self.input = {'Volume': volume, 'Surface': surface, 'Absorption coefficient': alpha}
        self.volume = volume
        self.surface = surface
        self.sub_surface = sub_surface
        self.alpha = alpha
        self.sub_alpha = sub_alpha
        self.use = use
        self.ErrorMessage = []

    def criticalDistance(self):
        '''
        Function to calculate the critical distance, where the energy densities of the direct and reflected soundfield are equal. 
        Calculation is made with an approximate formula based on statistical acoustics in a diffuse soundfield.
        '''
        
        criticalDistance = np.sqrt(self.equivalentAbsorptionSurface() / 50)

        return criticalDistance
        
    def equivalentAbsorptionSurface(self):
        '''Function to calculate the equivalent absorption surface.'''

        equivalentAbsorptionSurface = basic_dict()

        wall_index = 0
        # equivalent absorption surface for main walls 
        for octaveBands in self.alpha:
            alphaList = self.alpha[octaveBands]

            for walls in self.surface:
                equivalentAbsorptionSurface[octaveBands] = equivalentAbsorptionSurface[octaveBands] + (self.surface[walls] - sum(self.sub_surface[walls])) * alphaList[wall_index]
                wall_index = wall_index + 1
            wall_index = 0

        # adding equivalent absorption surface for sub walls
        for octaveBands in self.sub_alpha.keys():
            sub_alphaDict = self.sub_alpha[octaveBands]
            for sub_walls in self.sub_surface.keys():
                sub_alphaList = sub_alphaDict[sub_walls]
                sub_surfaceList = self.sub_surface[sub_walls]
                    
                for sub_wall_index in range(len(sub_surfaceList)):
                    equivalentAbsorptionSurface[octaveBands] = equivalentAbsorptionSurface[octaveBands] + sub_surfaceList[sub_wall_index] * sub_alphaList[sub_wall_index]

        return equivalentAbsorptionSurface
    
    
    def equivalentAbsorptionSurface_people(self, numberOfPeople, peopleDescription):
        '''
        Function to add equivalent absorption surface based on the number of people in the room and their specification regarding age and position (e.g. standing, sitting).
        Data retrieved from Table A.1 in DIN 18041
        '''
        equivalentAbsorptionSurface_total = basic_dict()
        equivalentAbsorptionSurface_walls = self.equivalentAbsorptionSurface()

        equivalentAbsorptionSurface_people = read_db('equivalentAbsorptionSurface_people_data.csv')

        equivalentAbsorptionSurface_people_list =  equivalentAbsorptionSurface_people[peopleDescription]

        index = 0

        for octaveBands in equivalentAbsorptionSurface_walls:
            
            equivalentAbsorptionSurface_total[octaveBands] = equivalentAbsorptionSurface_walls[octaveBands] + numberOfPeople * equivalentAbsorptionSurface_people_list[index]
            index += 1
            
        return equivalentAbsorptionSurface_total
   
    def reverberationTime(self):
        '''Function to calculate the reverberation time.'''

        reverberationTimeSeconds = basic_dict()
        equivalentSurface = self.equivalentAbsorptionSurface()
        for octavebands in equivalentSurface:
            reverberationTimeSeconds[octavebands] = (self.volume / equivalentSurface[octavebands]) * 0.161

        return reverberationTimeSeconds
    
    def reverberationTime_ratio(self):
        '''Function to calculate the ratio of given reverberation time to wanted reverberation time. Wanted reverberation time is based on the rooms use case and its volume.'''

        reverberationTime_ratio = basic_dict()
        ReverberationTime_upperlimit = {'125 Hz':1.45 , '250 Hz':1.2 , '500 Hz':1.2 , '1 kHz':1.2, '2 kHz':1.2 , '4 kHz':1.2 }
        ReverberationTime_lowerlimit = {'125 Hz':0.65 , '250 Hz':0.8 , '500 Hz':0.8 , '1 kHz':0.8, '2 kHz':0.8 , '4 kHz':0.65 }

        # Calculation of the wanted reverberation time dependent on the use case given in DIN 18041 (and the volume in use case "Sport")
        if self.use == 'Musik':
            reverberationTime_wanted = 0.45 * math.log10(self.volume) + 0.07

        elif self.use == 'Sprache/Vortrag':
            reverberationTime_wanted = 0.37 * math.log10(self.volume) - 0.14

        elif self.use == 'Sprache/Vortrag inklusiv':    
            reverberationTime_wanted = 0.32 * math.log10(self.volume) - 0.17

        elif self.use == 'Unterricht/Kommunikation':
            reverberationTime_wanted = 0.32 * math.log10(self.volume) - 0.17

        elif self.use == 'Unterricht/Kommunikation inklusiv':
            reverberationTime_wanted = 0.26 * math.log10(self.volume) - 0.14

        elif self.use == 'Sport':
                if self.volume > 10000:        
                    reverberationTime_wanted = 2
                else:
                    reverberationTime_wanted = 0.75 * math.log10(self.volume) - 1

        # Calculation of the ratio of calculated reverberation time to wanted reverberation time from DIN 18041 
        for octaveBands in self.reverberationTime():
            reverberationTime_ratio[octaveBands] = self.reverberationTime()[octaveBands] / reverberationTime_wanted
            if reverberationTime_ratio[octaveBands] > ReverberationTime_upperlimit[octaveBands]:
                self.ErrorMessage.append(f'Nachhallzeit in Oktavband mit Mittenfrequenz {octaveBands} zu hoch')
            elif reverberationTime_ratio[octaveBands] < ReverberationTime_lowerlimit[octaveBands]:
                self.ErrorMessage.append(f'Nachhallzeit in Oktavband mit Mittenfrequenz {octaveBands} zu niedrig')

        return reverberationTime_ratio, self.ErrorMessage
    
    def plot_reverberationTime(self):
        '''Function, which returns a plot of the reverberation time in octave bands.'''

        freq = np.array([125,250,500,1000,2000,4000])
        reverberationTimeSeconds = self.reverberationTime()

        fig = go.Figure()

        trace1 = go.Bar(x = freq, 
                        y = list(reverberationTimeSeconds.values()), 
                        name = 'bar', 
                        marker_color = 'blue', 
                        showlegend= True, 
                        width=.2)
        
        fig.add_trace(trace1)

        fig.update_layout(xaxis_title = 'Frequenz [Hz]', 
                          yaxis_title = 'Nachhallzeit [s]', 
                          width = 1000, 
                          height = 600)
        
        fig.update_xaxes(type='category')

        return fig
    
    def plot_reverberationTime_ratio(self):
        '''
        Function, which returns a plot of the calculated reverberation time in comparison to the wanted reverberation time 
        and the allowed deviations in octave bands.'''
        
        frequencies = [125,250,500,1000,2000,4000]
        
        ReverberationTime_upperlimit = [1.45, 1.2, 1.2, 1.2, 1.2, 1.2]
        ReverberationTime_lowerlimit = [0.65, 0.8, 0.8, 0.8, 0.8, 0.65]

        reverberationTime_ratio = list(self.reverberationTime_ratio()[0].values())

        fig = go.Figure()

        trace1 = go.Scatter(x = frequencies, 
                            y = ReverberationTime_lowerlimit, 
                            name = 'Grenzen', 
                            marker_color = 'green', 
                            mode = 'lines', 
                            legendgroup = 'boundaries',
                            hoverinfo = 'skip'
                            )
        
        trace2 = go.Scatter(x = frequencies, 
                            y = ReverberationTime_upperlimit, 
                            marker_color = 'green', 
                            fill = 'tonexty', 
                            fillcolor = 'rgba(26, 199, 93, 0.1)',
                            mode = 'lines',
                            legendgroup = 'boundaries',
                            showlegend = False,
                            hoverinfo = 'skip',
                            )
        
        trace3 = go.Bar(x = frequencies, 
                        y = reverberationTime_ratio, 
                        name = 'Nachhallzeitenvergleich',
                        width = .2,
                        marker_color = 'rgba(28, 122, 255, 1)',
                        # hovertemplate = 'Mittenfrequenz: %{x} Hz<br>Nachhallzeitenvergleich: %{y:.2f}', 
                        )
        
        fig.update_xaxes(type = 'category')       

        fig.update_layout(xaxis_title = 'Frequenz [Hz]',
                          yaxis_title = 'T / T_soll', 
                          width = 1000, 
                          height = 600, 
                        #   legend={'traceorder':'normal'},
                        #   hoverlabel = dict(bgcolor = 'rgba(28, 122, 255, .4)')
                          )
        
        fig.add_trace(trace1)
        fig.add_trace(trace2)
        fig.add_trace(trace3)
        
        return fig

room1 = room(volume , surface, sub_surface, alpha, sub_alpha, use)

room1.equivalentAbsorptionSurface()

# room1.equivalentAbsorptionSurface_people(6, 'Person sitzend auf Leichtpolsterbestuhlung')
# room1.plot_reverberationTime_ratio()

fig = room1.plot_reverberationTime()
fig.show()

fig2 = room1.plot_reverberationTime_ratio()
fig2.show()
errorMessage = room1.reverberationTime_ratio()[1]
print(errorMessage)


['Nachhallzeit in Oktavband mit Mittenfrequenz 250 Hz zu hoch\n', 'Nachhallzeit in Oktavband mit Mittenfrequenz 500 Hz zu hoch\n', 'Nachhallzeit in Oktavband mit Mittenfrequenz 1 kHz zu hoch\n', 'Nachhallzeit in Oktavband mit Mittenfrequenz 2 kHz zu hoch\n', 'Nachhallzeit in Oktavband mit Mittenfrequenz 4 kHz zu hoch\n', 'Nachhallzeit in Oktavband mit Mittenfrequenz 250 Hz zu hoch\n', 'Nachhallzeit in Oktavband mit Mittenfrequenz 500 Hz zu hoch\n', 'Nachhallzeit in Oktavband mit Mittenfrequenz 1 kHz zu hoch\n', 'Nachhallzeit in Oktavband mit Mittenfrequenz 2 kHz zu hoch\n', 'Nachhallzeit in Oktavband mit Mittenfrequenz 4 kHz zu hoch\n']


In [89]:
(20-5-4)*0.2+(20-6-2)*0.1+(10-5)*0.2+10*0.1+8*0.1+(8-1-2-3)*0.1 + 5*0.15+4*0.36 + 6*0.16+2*0.12 + 5*0.13 + 1*.1+2*.2+3*.3 + 5*.1


12.340000000000002

In [81]:
# 4 kHz
(20-5-4)*.1+(20-6-2)*.1+5*.1+10*.1+8*.1+(8-1-2-3)*.1 + 5*.2+4*.1+6*.2+2*.1+5*.13+1*.1+2*.2+3*.3


9.650000000000002

In [83]:
#250
(20-5-4)*0.1+(20-6-2)*0.1+(10-5)*0.1+10*0.1+8*0.1+(8-1-2-3)*0.1 + 5*.2+4*.1+6*.3+2*.1+5*.13+1*.1+2*.2+3*.3

10.25